# **Dados do Indice IBOVESPA B3**

## **O problema**

Você foi recentemente alocado em uma equipe de cientistas de dados de um grande fundo de investimentos brasileiro. Sua missão inicial é desenvolver um modelo preditivo capaz de prever se o índice IBOVESPA vai fechar em alta ou baixa no dia seguinte, com base em dados históricos do próprio índice. Esse modelo será usado como insumo para alimentar dashboards internos de tomada de decisão dos analistas quantitativos da empresa.

### **Dados**

Utilize os dados históricos do índice IBOVESPA, disponíveis publicamente: https://br.investing.com/indices/bovespa-historical-data
Requisitos:  
• Selecione o período “diário”.  
• Baixe um intervalo de pelo menos 2 anos de dados.  
• Realize o pré-processamento necessário para utilizar os dados no modelo.

### **Caracteristicas (Colunas)**

**Data** - Data de cada de negociação do IBOVESPA  
**Último** - Fechamento da negociação diária  
**Abertura** - Inicio das negociações e onde a negociação abriu  
**Máxima** - Nível máximo que o IBOVESPA alcançou no dia  
**Mínimo** - Nível mínimo que o IBOVESPA alcançou no dia  
**Vol.** - Volume de negociação diária  
**Var%** - Variação percentual diária partindo de sua abertura

### **Objetivo**

Criar um modelo que preveja se o fechamento do IBOVESPA do dia seguinte será maior ou menor que o do dia atual, ou seja:
Tech challenge Página 3 de 5.  
Seu modelo deve prever a tendência (↑ ou ↓) com acuracidade mínima de 75% em um conjunto de teste. O conjunto de testes deverá conter o último mês (30 dias) de dados disponíveis.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import(
    accuracy_score,
    precision_score,
    recall_score,
    confusion_matrix
)

sns.set_style()

In [10]:
url = "https://raw.githubusercontent.com/JacksonvBarbosa/Analise_Indice_Ibovespa/refs/heads/Jackson/Dataset/Dados%20Hist%C3%B3ricos%20-%20Ibovespa.csv"

In [11]:
df_ibov = pd.read_csv(url)

In [12]:
df_ibov_copia = df_ibov.copy()
df_ibov_copia

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,04.06.2025,137.002,137.547,138.797,136.695,"10,20M","-0,40%"
1,03.06.2025,137.546,136.787,137.672,136.175,"9,70B","0,56%"
2,02.06.2025,136.787,137.026,138.471,136.483,"9,10B","-0,18%"
3,30.05.2025,137.027,138.546,138.637,136.726,"15,14B","-1,09%"
4,29.05.2025,138.534,138.869,139.108,137.993,"8,83B","-0,25%"
...,...,...,...,...,...,...,...
597,10.01.2023,110.817,109.129,111.193,108.478,"13,70M","1,55%"
598,09.01.2023,109.130,108.964,109.938,108.134,"12,05M","0,15%"
599,06.01.2023,108.964,107.642,109.433,107.642,"12,63M","1,23%"
600,05.01.2023,107.641,105.336,107.743,105.333,"15,51M","2,19%"


In [22]:
display(df_ibov_copia.info())
display(f'Tamanho do dataset: linhas/coluna{df_ibov_copia.shape}')
display(df_ibov_copia.describe())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 602 entries, 2025-06-04 to 2023-01-04
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Último      602 non-null    float64
 1   Abertura    602 non-null    float64
 2   Máxima      602 non-null    float64
 3   Mínima      602 non-null    float64
 4   Vol.        602 non-null    float64
 5   Var%        602 non-null    float64
 6   Fechamento  602 non-null    Int64  
dtypes: Int64(1), float64(6)
memory usage: 38.2 KB


None

'Tamanho do dataset: linhas/coluna(602, 7)'

,Último,Abertura,Máxima,Mínima,Vol.,Var%,Fechamento
count,602.000000,602.000000,602.000000,602.000000,6.020000e+02,602.000000,602.0
mean,122.679924,122.625791,123.474872,121.867294,1.836857e+11,0.050166,0.490033
std,9.312727,9.324912,9.257544,9.368049,4.199744e+11,0.966530,0.500316
min,97.926000,97.926000,99.258000,96.997000,4.330000e+08,-3.150000,0.0
25%,117.068500,117.031250,117.960500,116.177000,9.202500e+08,-0.547500,0.0
50%,125.367500,125.293000,126.206000,124.567000,1.110500e+09,0.030000,0.0
75%,129.166500,129.125000,129.852250,128.209000,1.488500e+09,0.650000,1.0
max,140.110000,140.109000,140.382000,138.966000,2.487000e+12,4.290000,1.0


### Observando as informações dos dados, para melhor manipulação futura devemos converter a coluna **Data** para DateTime e a coluna **Vol.** e **Var%** para Float

1. Qual é a média de fechamento nos ultimos 2 ano?  
* 122 mil pontos  
2. O quanto os pontos estão dispersos da média?  
* mais ou menos 9.3mil pontos
3. Quais foi a mínima e a máxima em pontos que o IBOVESPA alcançou nesses 2 ano?  
* **minima** de 97.926 pontos e uma **máxima** de 140.11 pontos  
4. A mediana está próxima da média de pontos?  
* Sim a **mediana** mais ou menos 3 mil pontos ficando dentro do desvio padrão de fechamentodo indice  


### Uma análise básica exploratória, como uma **mediana** próxima da média nos mostrando que não tivemos umas dispersão significativa vamos ver mais a frente mais da para perceber que não tivemos um outlier nesse periodo

In [14]:
# Formatando caracteristica Data e Convertendo em datetime
df_ibov_copia['Data'] = pd.to_datetime(df_ibov_copia['Data'], format = '%d.%m.%Y').dt.strftime('%d/%m/%Y')
df_ibov_copia['Data'] = pd.to_datetime(df_ibov_copia['Data'], format='%d/%m/%Y')

In [15]:
# Função para conversão dos dados de Volume de string para float
## OBS.: Caso acertando com o grupo e vermos a necessidade de mudar par outro tipo fazer a mudança
def converter_volume(vol):
    if pd.isna(vol):
        return np.nan
    vol = vol.upper().replace(',', '').strip()
    if vol.endswith('B'):
        return float(vol[:-1]) * 1e9
    elif vol.endswith('M'):
        return float(vol[:-1]) * 1e6
    elif vol.endswith('K'):
        return float(vol[:-1]) * 1e3
    else:
        return float(vol)
        
df_ibov_copia['Vol.'] = df_ibov_copia['Vol.'].apply(converter_volume)

In [16]:
df_ibov_copia['Var%'] = df_ibov_copia['Var%'].str.replace('%', '').str.replace(',', '.').astype(float)

In [17]:
print(df_ibov_copia[['Vol.', 'Var%']].dtypes)
print(df_ibov_copia[['Vol.', 'Var%']].head())

Vol.    float64
Var%    float64
dtype: object
           Vol.  Var%
0  1.020000e+09 -0.40
1  9.700000e+11  0.56
2  9.100000e+11 -0.18
3  1.514000e+12 -1.09
4  8.830000e+11 -0.25


### Criar uma caracteristica (coluna) de fechamento positivo(1) ou negativo(0) utilizando a caracteristica [Último] como referência

In [ ]:
'''Exemplo do metodo Shift ele irá varrer o data frame e conforme o parametro colocado nele ele irá fazer o deslocamento
para linha seguinte ou anterior
Ex.: Shift(n) -> Compara valores anteriores (n>0)
     Shift(n) -> Compara valores seguintes (n<0) Vamos usar este para buscar valores seguintes
     Shift(n) -> Compara com o dia anterior (n=1)

''' 
# Exemplos de uso
df = pd.DataFrame({'Preço': [100, 102, 101, 105]})
df['Anterior'] = df['Preço'].shift(1)
df['Seguinte'] = df['Preço'].shift(-1)

df

## Não tem a necessidade de Rodar esse código

,Preço,Anterior,Seguinte
0,100,NaN,102.0
1,102,100.0,101.0
2,101,102.0,105.0
3,105,101.0,NaN


In [ ]:
# Caso quisermos utilizar o loop for

'''fechamento = df_ibov['Último'].values
fechamento_categoria = []

for i in range(len(fechamento) - 1):
    if fechamento[i + 1] > fechamento[i]:
        fechamento_categoria.append(1)
    else:
        fechamento_categoria.append(0)

# Como comparamos até o penúltimo, o último dia não tem "dia seguinte"
fechamento_categoria.append(None)  # ou np.nan se preferir

# Adiciona a nova coluna no DataFrame
#df_ibov['Subiu_dia_seguinte'] = fechamento_categoria

print(fechamento_categoria)'''

[1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 

In [18]:
# Cria a coluna fechamento com valores Binarios 0 para negativo e 1 para positivo
df_ibov_copia['Fechamento'] = (df_ibov['Último'].shift(-1) > df_ibov['Último']).astype('Int64')

In [19]:
# Setando data como indice ()index
df_ibov_copia = df_ibov_copia.set_index('Data')
df_ibov_copia

,Último,Abertura,Máxima,Mínima,Vol.,Var%,Fechamento
Data,,,,,,,
2025-06-04,137.002,137.547,138.797,136.695,1.020000e+09,-0.40,1
2025-06-03,137.546,136.787,137.672,136.175,9.700000e+11,0.56,0
2025-06-02,136.787,137.026,138.471,136.483,9.100000e+11,-0.18,1
2025-05-30,137.027,138.546,138.637,136.726,1.514000e+12,-1.09,1
2025-05-29,138.534,138.869,139.108,137.993,8.830000e+11,-0.25,1
...,...,...,...,...,...,...,...
2023-01-10,110.817,109.129,111.193,108.478,1.370000e+09,1.55,0
2023-01-09,109.130,108.964,109.938,108.134,1.205000e+09,0.15,0
2023-01-06,108.964,107.642,109.433,107.642,1.263000e+09,1.23,0


In [21]:
# Existe dados Nulos
df_ibov_copia.isnull().sum()

Último        0
Abertura      0
Máxima        0
Mínima        0
Vol.          0
Var%          0
Fechamento    0
dtype: int64

In [23]:
# Inicio e fim do ano de extraidos do dataset
print(f'Data Inicial: {df_ibov_copia.index.min()}')
print(f'Data Inicial: {df_ibov_copia.index.max()}')

Data Inicial: 2023-01-04 00:00:00
Data Inicial: 2025-06-04 00:00:00


In [26]:
# Temos dados duplicados
df_ibov_copia.duplicated().sum()

np.int64(0)